In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

# 每个批次的大小
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)   #生成一个截断的正态分布
    return tf.Variable(initial)

#初始化偏置
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# 卷积层
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

#池化层
def max_pool_2x2(x):
    #ksize [1,x,y,1]  窗口
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 定义两个placeholder
x = tf.placeholder("float", [None, 784])   #28*28
y = tf.placeholder("float", [None, 10])

#改变x的格式转为4D的向量[batch, in_height, in_width, in_channels]
x_image = tf.reshape(x, [-1, 28, 28, 1])

#初始化第一个卷积层的权值和偏置
W_conv1 = weight_variable([5,5,1,32])  #5*5的采样窗口，32个卷积核从 1 个平面抽取特征。  1个平面即 同x_image中1，因为黑白
b_conv1 = bias_variable([32])    #每一个卷积核一个偏置值

#把x_image 和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)   # 进行max-pooling

#初始化第二个卷积层的权值和偏置
W_conv2 = weight_variable([5, 5, 32, 64])  # 64个卷积核从32个平面抽取特征
b_conv2 = bias_variable([64])

#把h_pool1 和权值向量进行卷积
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2) 

#28*28 的图片第一次卷积后还是28*28，第一次池化后变为14*14
#第二次卷积后为14*14，第二次池化后变为7*7
#经过上面操作后得到64 张 7*7的特征平面

#初始化第一个全连接层的权值
W_fe1 = weight_variable([7*7*64, 1024])  #上一层有7*7*64个神经元，全连接层有1024个神经元
b_fe1 = bias_variable([1024])  #1024个节点

#把池化层2的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
#求第一个全连接层的输出
h_fe1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fe1) + b_fe1)

# keep_prob 用来表示神经元的输出概率
keep_prob = tf.placeholder("float")
h_fe1_drop = tf.nn.dropout(h_fe1, keep_prob)

#初始化第二个全连接层
W_fe2 = weight_variable([1024, 10])
b_fe2 = bias_variable([10])


#计算输出
prediction = tf.nn.softmax(tf.matmul(h_fe1_drop, W_fe2) + b_fe2)

 # 使用交叉熵的方法改进 #########################################################
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))       
# 使用AdamOptimizer 进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# 结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))      # argmax 返回一维变量中最大值所在位置
# 求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict = {x:batch_xs, y:batch_ys, keep_prob:0.7})
            
        acc = sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
        print("Iter" + str(epoch) + ", Test Accuracy" + str(acc))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Iter0, Test Accuracy0.8643
Iter1, Test Accuracy0.8771
Iter2, Test Accuracy0.9778
Iter3, Test Accuracy0.9822
Iter4, Test Accuracy0.9835
Iter5, Test Accuracy0.9839
Iter6, Test Accuracy0.9867
Iter7, Test Accuracy0.9868
Iter8, Test Accuracy0.987


KeyboardInterrupt: 